## Load mesh

In [1]:
import openmesh as om
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt

# mesh = om.read_trimesh('foam15.off')
mesh = om.read_trimesh('siete_nodos.off')
mesh = om.read_trimesh('dm4.off')


## Generate initial setup

In [2]:
V=mesh.points()
def get_T(x,y):
    return x*(1+0.5*(1-y)**2)

def get_S(x,y):
    return x*(1-y)

X=V[:,0]
Y=V[:,1]

T = get_T(V[:,0], V[:,1])
S = get_S(V[:,0], V[:,1])
U = np.array([*T, *S])

In [3]:
## Rutines to calculate gradients


In [4]:
import math

def getGrad2(I, U):
    m = np.zeros((2, 2))
    b = np.zeros((2, 1))

    for k in range(2):
        m[k,0] = V[I[k+1],0]-V[I[0],0]
        m[k,1] = V[I[k+1],1]-V[I[0],1]
        b[k] = U[k+1]-U[0]
                
    g = np.linalg.solve(m, b)    
    
    return g

def getGradients(vh, W):
    nVertex = vh.idx()
    
    vlist = []
    nlist = []    
    
    for vvh in mesh.vv(vh):
        idx = vvh.idx() # devuelve indice del VERTICE
        vlist.append(idx)
        nlist.append(V[nVertex,:]-V[idx,:])
    nlist=np.array(nlist)
    # print(nVertex,': \n', nlist,'\n')
    
    # for n in nlist:
    THETA=[]
    GRADIENT=[]
    for k in range(len(vlist)):
            v1 = nlist[k-1,:]
            v2 = nlist[k,:]
            theta = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
            I=[nVertex, vlist[k-1], vlist[k]]                
                
            if (abs(theta)<math.pi-1e-3) and (abs(theta) > 1e-3) and (abs(theta-math.pi/2)>1e-3):
                g = getGrad2(I, W[I])
                THETA.append(theta)
                GRADIENT.append(g)
                
    THETA=np.array(THETA)  
    GRADIENT=np.array(GRADIENT)

    return THETA, GRADIENT

for vh in mesh.vertices():
    k = vh.idx()
    # if (k>20) and (k < 30):
    #    print(k)
        
    if k > 45:
        break

print(k)

print('nVertex:', vh.idx())
theta, gradient = getGradients(vh, T)

print('\n sum(', theta,') = % .2f' % sum(theta),'\n \n')
print('GRADIENT: \n', gradient.transpose(),'\n \n')

46
nVertex: 46

 sum( [0.19739558 1.37340075 1.37340075 0.19739558 1.37340076 0.19739556] ) =  4.71 
 

GRADIENT: 
 [[[ 1.405       1.36124999  1.36124999  1.405       1.45125
    1.405     ]
  [-1.09375    -1.31249999 -1.31249999 -1.53124999 -1.3875
   -1.15625   ]]] 
 



In [7]:
import math

## Residual
def residual(U):
    N = int(len(U)/2)
    T = U[0:N]
    S = U[N:]
    r = []
    
    for vh in mesh.vertices():
        k = vh.idx()            
            
        # calculate gradients
        theta, gT = getGradients(vh,T)
        theta, gS = getGradients(vh,S)

        # print('gS, gT', theta, gS, gT, '\n ....... \n')            

        if len(gT) == 0:
            print('strange vertex:', k)
            for vvh in mesh.vv(vh):
                print('\t', vvh.idx())
        else:
            # calculate the Hamiltonians
            gT_av = gT.transpose().dot(theta)/sum(theta+1e-5)
            gS_av = gS.transpose().dot(theta)/sum(theta+1e-5)            
            
            # residual of T
            rT = np.linalg.norm(gT_av) - V[k,1]/(S[k] + 1e-5)
            rS = gT_av/np.linalg.norm(gT_av+1e-5) - gS_av  
            
            # print('gSav, gTav', gS_av, gT_av, '\n ....... \n')
            # print('..-----------------------------------------------------------------------')

            rS=rS[0] # strange reformatting needed
            
            r.append(rT)
            r.append(rS[0])
            r.append(rS[1])
            
        if V[k,0] == 0:
            r.append(S[k])
            r.append(T[k])
        elif V[k,1] == 1:
            r.append(T[k]-V[k,0])
            r.append(S[k])
                
            # print(rS, type(rS), '\n ....... \n')
            # print(rS[0])
            # print(rS[1])
            
            # r[k*3] = rT
            # r[k*3+1] = rS[0]
            # r[k*3+2] = rS[1]
                
            # r[nVertex] = rT
            # r[k+N] = 
            
        #
        #
        # print('k =', k, ', \t r :', r)
        # 
    # print(k)
    r=np.array(r)
        
    return r


def cost(u):
    R=residual(u)
    c=np.inner(R,R)
    return c

print('---- > --------------------------------------')
res = residual(U)
print(res)
print('------ < ------------------------------------')

---- > --------------------------------------
strange vertex: 1
	 0
	 21
	 2
strange vertex: 5
	 4
	 25
	 6
strange vertex: 9
	 8
	 29
	 10
strange vertex: 11
	 10
	 31
	 12
strange vertex: 16
	 15
	 36
	 17
strange vertex: 18
	 17
	 38
	 19
strange vertex: 20
	 40
	 21
	 0
strange vertex: 27
	 28
	 7
	 26
	 47
strange vertex: 30
	 50
	 31
	 10
	 29
strange vertex: 39
	 19
	 38
	 59
strange vertex: 41
	 42
	 21
	 40
	 61
strange vertex: 45
	 65
	 46
	 25
	 44
strange vertex: 53
	 54
	 33
	 52
	 73
strange vertex: 60
	 80
	 61
	 40
strange vertex: 63
	 83
	 64
	 43
	 62
strange vertex: 66
	 86
	 67
	 46
	 65
strange vertex: 68
	 69
	 48
	 67
	 88
strange vertex: 72
	 73
	 52
	 71
	 92
strange vertex: 81
	 101
	 82
	 61
	 80
strange vertex: 95
	 94
	 115
	 96
	 75
strange vertex: 97
	 96
	 117
	 98
	 77
strange vertex: 100
	 120
	 101
	 80
strange vertex: 105
	 85
	 104
	 125
	 106
strange vertex: 108
	 128
	 109
	 88
	 107
strange vertex: 110
	 111
	 90
	 109
	 130
strange vertex: 114
	

In [49]:
import sys

print('pipe screen output into file...')
sys.stdout = open('output.txt', 'w') 

pipe screen output into file...


## Run optimization routine

In [6]:
import scipy.optimize as opt
import time
import sys

print('pipe screen output into file...')
sys.stdout = open('output.txt', 'w') 

for method in ('nelder-mead', 'powell', 'cobyla','L-BFGS-B','BFGS'):
    t0 = time.time()
    res = opt.minimize(cost, U, method=method, tol=1e-10, options={'maxiter': 2, 'disp': True})
    t1 = time.time()
    
    # U = res.x
    print(res)
    print('\n<<< ---------',k,'----  %s' % method, ' ------ dt = %.1f \n\n\n' % float(t1 - t0))

pipe screen output into file...
